In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from pydataset import data

In [60]:
# Attendance Data
# Load the attendance.csv file and calculate an attendnace percentage for each student. 
# One half day is worth 50% of a full day, and 10 tardies is equal to one absence.

attendance_df = pd.read_csv('untidy-data/attendance.csv')

In [61]:
attendance_df = attendance_df.rename(columns={"Unnamed: 0": "Student"})

In [62]:
tidy_attendance = attendance_df.melt(id_vars=['Student'], var_name='status')

In [63]:
for i in range(len(tidy_attendance)):
    if tidy_attendance.value[i] == 'P':
        tidy_attendance.value[i] = float(1)
    elif tidy_attendance.value[i] == 'H':
        tidy_attendance.value[i] = float(.5)
    elif tidy_attendance.value[i] == 'T':
        tidy_attendance.value[i] = float(.9)
    elif tidy_attendance.value[i] == 'A':
        tidy_attendance.value[i] = float(0)
    else:
        print("There is a glitch in the matrix")

In [64]:
tidy_attendance = tidy_attendance.astype({'value': 'float'})

In [65]:
tidy_attendance.groupby('Student').value.mean()

Student
Billy    0.5250
Jane     0.6875
John     0.9125
Sally    0.7625
Name: value, dtype: float64

In [66]:
tidy_attendance.value.dtype

dtype('float64')

In [69]:
# Read the coffee_levels.csv file.
coffee_df = pd.read_csv('untidy-data/coffee_levels.csv')

,hour,coffee_carafe,coffee_amount
0,8,x,0.816164
1,9,x,0.451018
2,10,x,0.843279
3,11,x,0.335533
4,12,x,0.898291


In [70]:
coffee_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   hour           30 non-null     int64  
 1   coffee_carafe  30 non-null     object 
 2   coffee_amount  30 non-null     float64
dtypes: float64(1), int64(1), object(1)
memory usage: 848.0+ bytes


In [71]:
coffee_df.value_counts()

hour  coffee_carafe  coffee_amount
8     x              0.816164         1
      y              0.189297         1
17    y              0.594126         1
      x              0.391560         1
16    z              0.280621         1
      y              0.544676         1
      x              0.183891         1
15    z              0.436364         1
      y              0.144644         1
      x              0.215043         1
14    z              0.864464         1
      y              0.058361         1
      x              0.507288         1
13    z              0.398520         1
      y              0.997464         1
      x              0.310711         1
12    z              0.771947         1
      y              0.017009         1
      x              0.898291         1
11    z              0.311495         1
      y              0.235529         1
      x              0.335533         1
10    z              0.144928         1
      y              0.023163         1
     

In [72]:
# Transform the data so that each carafe is in it's own column.
coffee_pivot = coffee_df.pivot_table(index = ['hour'],
                    columns = 'coffee_carafe',
                    values = 'coffee_amount').reset_index()

In [73]:
coffee_pivot

coffee_carafe,hour,x,y,z
0,8,0.816164,0.189297,0.999264
1,9,0.451018,0.521502,0.915990
2,10,0.843279,0.023163,0.144928
3,11,0.335533,0.235529,0.311495
4,12,0.898291,0.017009,0.771947
5,13,0.310711,0.997464,0.398520
6,14,0.507288,0.058361,0.864464
7,15,0.215043,0.144644,0.436364
8,16,0.183891,0.544676,0.280621
9,17,0.391560,0.594126,0.436677


In [74]:
# Read the cake_recipes.csv data. This data set contains cake tastiness scores for combinations 
# of different recipes, oven rack positions, and oven temperatures.

cake_df = pd.read_csv('untidy-data/cake_recipes.csv')

In [75]:
cake_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   recipe:position  8 non-null      object 
 1   225              8 non-null      float64
 2   250              8 non-null      float64
 3   275              8 non-null      float64
 4   300              8 non-null      float64
dtypes: float64(4), object(1)
memory usage: 448.0+ bytes


In [76]:
cake_df.value_counts()

recipe:position  225        250        275        300      
a:bottom         61.738655  53.912627  74.414730  98.786784    1
a:top            51.709751  52.009735  68.576858  50.228470    1
b:bottom         57.095320  61.904369  61.196980  99.248541    1
b:top            82.455004  95.224151  98.594881  58.169349    1
c:bottom         96.470207  52.001358  92.893227  65.473084    1
c:top            71.306308  82.795477  92.098049  53.960273    1
d:bottom         52.799753  58.670419  51.747686  56.183110    1
d:top            96.873178  76.101363  59.571620  50.971626    1
dtype: int64

In [79]:
# Tidy the data as necessary.
cake_df['recipe:position'].str.split(":", expand = True)
cake_df[['recipe', 'position']] = cake_df['recipe:position'].str.split(':', expand = True)
cake_df.drop(columns = 'recipe:position', inplace = True)

In [80]:
# Melt data
cake_df_melt = cake_df.melt(id_vars = ['recipe', 'position'], var_name = 'temperature', value_name = 'score')
cake_df_melt.head()

,recipe,position,temperature,score
0,a,bottom,225,61.738655
1,a,top,225,51.709751
2,b,bottom,225,57.095320
3,b,top,225,82.455004
4,c,bottom,225,96.470207


In [83]:
# Which recipe, on average, is the best?
cake_df_melt.groupby(['recipe']).score.mean()
best_recipe = cake_df_melt.groupby(['recipe']).score.mean().idxmax()
best_score = cake_df_melt.groupby(['recipe']).score.mean().max()
print('The best recipe is', best_recipe, 'with score of', round(best_score, 2))

The best recipe is b  with score of 76.7


In [84]:
# Which oven temperature, on average, produces the best results?
cake_df_melt.groupby('temperature').score.mean()
best_temp = cake_df_melt.groupby(['temperature']).score.mean().idxmax()
best_score = cake_df_melt.groupby(['temperature']).score.mean().max()
print('The best temperature is', best_temp, 'with score of', round(best_score, 2))

The best temperature is 275 with score of 74.89


In [87]:
# Which combination of recipe, rack position, and temperature gives the best result?
cake_df_melt.groupby(['temperature', 'recipe', 'position']).score.mean()
best_comb = cake_df_melt.groupby(['temperature', 'recipe', 'position']).score.mean().idxmax()
best_score = cake_df_melt.groupby(['temperature', 'recipe', 'position']).score.mean().max()
print('The best combination is', best_comb, 'with score of', round(best_score, 2))

The best combination is ('300', 'b', 'bottom') with score of 99.25
